In [1]:
import cv2
import glob
import math
import pandas as pd
from ultralytics import YOLO
from coco_keypoints import COCOKeypoints

model = YOLO("../yolo11n-pose.pt")
fall_images = glob.glob("../fall_dataset/images/fall/*.jpg")
not_fall_images = glob.glob("../fall_dataset/images/not-fall/*.jpg")

def append_dataset(df, input_images_path, fall_condition: bool):
    for image_path in input_images_path:
        img = cv2.imread(image_path)
        img = cv2.resize(img, (640, 640))
        results = model(img)

        keypoint = results[0].keypoints.data[0].numpy()
        if len(keypoint) == 0:
            continue

        keypoints = COCOKeypoints.from_list(keypoint.flatten().tolist())

        len_factor = math.sqrt(
            ((keypoints.left_shoulder_y - keypoints.left_hip_y) ** 2 + 
             (keypoints.left_shoulder_x - keypoints.left_hip_x) ** 2)
        )

        data = {
            'left_shoulder_y': keypoints.left_shoulder_y,
            'left_shoulder_x': keypoints.left_shoulder_x,
            'right_shoulder_y': keypoints.right_shoulder_y,
            'right_shoulder_x': keypoints.right_shoulder_x,
            'left_body_y': keypoints.left_hip_y,
            'left_body_x': keypoints.left_hip_x,
            'right_body_y': keypoints.right_hip_y,
            'right_body_x': keypoints.right_hip_x,
            'len_factor': len_factor,
            'left_knee_y': keypoints.left_knee_y,
            'left_knee_x': keypoints.left_knee_x,
            'right_knee_y': keypoints.right_knee_y,
            'right_knee_x': keypoints.right_knee_x,
            'left_foot_y': keypoints.left_ankle_y,
            'right_foot_y': keypoints.right_ankle_y,
            'fall': fall_condition
        }
        temp = pd.DataFrame(data, index=[0])
        df = pd.concat([df, temp], ignore_index=True)

    return df

In [2]:
df = pd.DataFrame(columns=['left_shoulder_y', 'left_shoulder_x', 'right_shoulder_y', 'right_shoulder_x', 'left_body_y', 'left_body_x', 'right_body_y', 'right_body_x', 'len_factor', 'left_knee_y', 'left_knee_x', 'right_knee_y', 'right_knee_x', 'left_foot_y', 'right_foot_y', 'fall'])

df = append_dataset(df, fall_images, True)
df = append_dataset(df, not_fall_images, False)

print(f"dropping {sum((df == 0).sum(axis=1) > 5)} rows")
dataset = df[(df == 0).sum(axis=1) <= 5]
dataset.reset_index(drop=True, inplace=True)

dataset.head()
dataset.to_csv('../ml/clean_data.csv', index=False)
print(f"saved {len(dataset)} rows to clean_data.csv")


0: 640x640 1 person, 70.5ms
Speed: 1.5ms preprocess, 70.5ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 65.1ms
Speed: 1.4ms preprocess, 65.1ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 68.4ms
Speed: 1.1ms preprocess, 68.4ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)



/var/folders/q3/rfd50l751570rjpc5f926_kc0000gn/T/ipykernel_23176/3411352820.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, temp], ignore_index=True)


0: 640x640 (no detections), 72.9ms
Speed: 1.1ms preprocess, 72.9ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 77.7ms
Speed: 1.3ms preprocess, 77.7ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 67.6ms
Speed: 1.2ms preprocess, 67.6ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 66.0ms
Speed: 0.9ms preprocess, 66.0ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 66.5ms
Speed: 1.1ms preprocess, 66.5ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 67.0ms
Speed: 1.2ms preprocess, 67.0ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 68.4ms
Speed: 1.1ms preprocess, 68.4ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 67.0ms
Speed: 1.3ms preprocess, 67.0ms inference, 0.6ms postprocess per image at shape (1